In [3]:
import csv
import vincent
import json
import pandas as pd
import numpy as np

vincent.initialize_notebook()

states = {'ak' : [0,0] , 'al' : [0,0] , 'ar' : [0,0] , 'az' : [0,0] , 'ca' : [0,0] , 'co' : [0,0] , 'ct' : [0,0] , 'de' : [0,0] , 'fl' : [0,0] , 'ga' : [0,0] , 'hi' : [0,0] , 'ia' : [0,0] , 'id' : [0,0] , 'il' : [0,0] , 
          'in' : [0,0] , 'ks' : [0,0] , 'ky' : [0,0] , 'la' : [0,0] , 'ma' : [0,0] , 'md' : [0,0] , 'me' : [0,0] , 'mi' : [0,0] , 'mn' : [0,0] , 'mo' : [0,0] , 'ms' : [0,0] , 'mt' : [0,0] , 'nc' : [0,0] , 'nd' : [0,0] , 'ne' : [0,0] , 'nh' : [0,0] , 
          'nj' : [0,0] , 'nm' : [0,0] , 'nv' : [0,0] , 'ny' : [0,0] , 'oh' : [0,0] , 'ok' : [0,0] , 'or' : [0,0] , 'pa' : [0,0] , 'ri' : [0,0] , 'sc' : [0,0] , 'sd' : [0,0] , 'tn' : [0,0] , 'tx' : [0,0] , 'ut' : [0,0] , 'va' : [0,0] , 'vt' : [0,0] , 
          'wa' : [0,0] , 'wi' : [0,0] , 'wv' : [0,0] , 'wy' : [0,0]}

states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

data2=open('/home/jose/BigData/Python/Twitter/AFINN-111.txt')

texto1=data2.readlines()

abecedario={'a':0,'b':0,'c':0,'d':0,'e':0,'f':0,'g':0,'h':0,'i':0,'j':0,'k':0,'l':0,'m':0,'n':0,'o':0,'p':0,'q':0,'r':0,'s':0,'t':0,'u':0,'w':0,'x':0,'y':0,'z':0}

sentimientos={}

for linea in texto1:
    aux=linea.split("\t")
    aux1=aux[1].split("\n")
    aux1[0]=int(aux1[0])
    sentimientos[aux[0]]=aux1[0]

def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        if len(state) > 1:
            return state[1]

def isState(state):
    if state in states.keys():
        return True
    return False 

def limpiar(cadena):
    for letra in cadena:
        if letra not in abecedario.keys():
            cadena=cadena.replace(letra,"")
    return cadena

def readTweets():
    file = "/home/jose/BigData/Python/Twitter/twitts.txt"

    
    array = []
    with open(file, "r") as ins:

    #data1=open('/home/jose/Master/Twitter/Nou_Twitter/twitts.txt')
    #ins=data1.readlines()
    
        for line in ins:          
            if ( len(line)> 1): ## to avoid empty lines 
                data = json.loads(line)
                if "created_at" in data:
                    state = getState(data)
                    if isState(state):
                        states[state][0]=states[state][0]+1
                        if "text" in data:
                            frase=data["text"].split(" ")
                            for palabra in frase:
                                n_palabra=limpiar(palabra.lower())
                                if type(sentimientos.get(n_palabra)) == int:
                                    states[state][1]=states[state][1]+sentimientos.get(n_palabra)
                                    #print(limpiar(palabra.lower()))
                                    #print(palabra.lower())
    
    mapas={}
    
    for llaves,valores in states.items():
        if valores[0]!=0:
            valores[1]=round(valores[1]/valores[0],2)
 #           states[llaves]=valores[1]
            mapas[states_names.get(llaves.upper())]=valores[1]
        else:
 #           states[llaves]=valores[1]
            mapas[states_names.get(llaves.upper())]=valores[1]
            
#    a_zonas=sorted(states,key=states.__getitem__,reverse=True)
    
#    zonas_ordenadas=[]
    
#    for zona in a_zonas:
#        zonas_ordenadas.append(states_names.get(zona.upper()))
        
    return mapas
    
   

In [4]:
mapa=readTweets()

In [5]:
felicidad = pd.DataFrame({'country_names' : np.array(list(mapa.keys())),
                                 'my_rate' : np.array(list(mapa.values()))})

state_topo = r'us_states.topo.json'

geo_data = [{'name': 'states',
             'url': state_topo,
             'feature': 'us_states.geo'}]

vis = vincent.Map(data=felicidad, geo_data=geo_data, scale=1000,
                  projection='albersUsa', data_bind='my_rate', data_key='country_names',
                  map_key={'states': 'properties.NAME'}, brew='RdPu')

vis.marks[0].properties.enter.stroke_opacity = vincent.ValueRef(value=0.5)

#Custom threshold scale
vis.scales[0].type='threshold'
vis.scales[0].domain = [-1.67,-1.15,-0.62,-0.10,0.43,0.95,1.48,2.00]
vis.legend(title='Grado de felicidad (mínimo es -5 y el máximo es 5)')
vis.to_json('vega.json')

vis.display()